In [1]:
import pandas as pd

Загрузим табличку с нашими данными про длительность поездок такси! 

In [2]:
df = pd.read_csv('taxi_dataset_with_target.csv', index_col=0)

df.head()

,vendor_id,pickup_datetime,passenger_count,store_and_fwd_flag,trip_duration,distance_km
id,,,,,,
id2875421,1,2016-03-14 17:24:55,930.399753,0,455.0,1.500479
id2377394,0,2016-06-12 00:43:35,930.399753,0,663.0,1.807119
id3858529,1,2016-01-19 11:35:24,930.399753,0,2124.0,6.392080
id3504673,1,2016-04-06 19:32:31,930.399753,0,429.0,1.487155
id2181028,1,2016-03-26 13:30:55,930.399753,0,435.0,1.189925


In [3]:
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])

In [4]:
df.dtypes

vendor_id                      int64
pickup_datetime       datetime64[ns]
passenger_count              float64
store_and_fwd_flag             int64
trip_duration                float64
distance_km                  float64
dtype: object

Попробуем построить модель Линейной Регрессии!

В качестве Выборки используйте таргетную колонку (*trip_duration*) и все признаки, кроме времени начала заказа (*pickup_datetime*)

Для начала воспользуемся "коробочным решением". Для этого создайте переменную model, положите в нее класс **LinearRegression** из модуля **linear_model** библиотеки **sklearn**.

Далее воспользуйтесь методом **fit**, положите в него через запятую: массив **X**, состоящий из признаков объектов (можно pandas dataframe'ом, а можно numpy-массивом), и массив **Y** с таргетами.

In [5]:
from sklearn.linear_model import LinearRegression

X = df.drop(['trip_duration', 'pickup_datetime'], axis=1)
Y = df['trip_duration']

In [6]:
model = LinearRegression()

model.fit(X, Y)

LinearRegression()

Чтобы посмотреть на значения полученных коэффициентов модели, следует обратиться к атрибуту **coef_** класса линейной регрессии. Для просмотра свободного веса, к атрибуту **intercept_** 

In [7]:
print(model.coef_.round(3))
print(model.intercept_.round(3))

[198.463   0.296  56.469 115.274]
171.657


Теперь реализуйте функцию LinearRegressionByMatrix, которая будет принимать на вход 3 параметра:

Матрицу объект-признак **(X)**, вектор с ответами **(Y)**, булевый параметр **fit_intercept**, цель которого - добавить константный признак (состоящий из единичек), если True, и ничего не делать, если False.

Функция должна вернуть одномерный np.array объект с оцененными **$\beta_1, ..., \beta_n, \beta_0$**

Реализуйте решение через аналитическое решение в матрицах из лекции и практики:

$$
\beta^* = (X^T \cdot X)^{-1} \cdot X^T \cdot Y
$$

In [8]:
import numpy as np


def LinearRegressionByMatrix(X: np.array, Y: np.array, fit_intercept: bool = True):
    """
    :param X: матрица объектов
    :param Y: вектор (матрица с 1 столбцом) ответов
    :param fit_intercept: добавить ли константную переменную в данные?

    :return: одномерный numpy-array с полученными коэффициентами beta
    """
    np.linalg.inv(np.dot(X.T, X))

    if fit_intercept:
        col_to_add = np.ones(len(X))
        X = np.column_stack((X, col_to_add))

    result = np.dot(np.dot(np.linalg.inv(np.dot(X.T, X)), X.T), Y)

    return result

In [9]:
X_train = df.drop(['trip_duration', 'pickup_datetime'], axis=1).values
Y_train = df['trip_duration'].values

result_coef = LinearRegressionByMatrix(X_train, Y_train)

In [10]:
result_coef.round(3)

array([198.463,   0.296,  56.469, 115.274, 171.657])

**Полученные коэффициенты совпадают с коэффициентами, полученными в `LinearRegression` из `sklearn.linear_model`.**

Получились ли коэффициенты такими же, как и в "коробочном" варианте?

Имея коэффициенты модели, можно восстановить предсказания для каждого объекта!

Сделайте это через операцию матричного произведения матрицы **X** и полученных коэффициентов (как результат работы *LinearRegressionByMatrix*)

In [11]:
col_to_add = np.ones(len(X_train))
X_train_with_one = np.column_stack((X_train, col_to_add))
Y_predict = np.dot(X_train_with_one, result_coef)

print(Y_predict)

[ 818.7747282   655.65912268 1382.6469154  ... 1548.74134353  573.4306718
  578.2338068 ]


Результат из `LinearRegression` из `sklearn.linear_model`:

In [12]:
Y_predict_skl = model.predict(X)

Видно, что результаты равны, проверим это, посчитав `MSE`, `RMSE`, `MAE` для двух моделей.

In [22]:
def MSE(Y, Y_predict):
    return ((Y - Y_predict) ** 2).mean()


def MAE(Y, Y_predict):
    return (abs(Y - Y_predict)).mean()


print(f'MSE для двух моделей (sklearn и LinearRegressionByMatrix) соответственно:\
      {MSE(Y, Y_predict_skl)}, {MSE(Y, Y_predict)}')

print(f'RMSE для двух моделей (sklearn и LinearRegressionByMatrix) соответственно:\
      {(MSE(Y, Y_predict_skl)) ** 0.5}, {(MSE(Y, Y_predict)) ** 0.5}')

print(f'MAE для двух моделей (sklearn и LinearRegressionByMatrix) соответственно:\
      {MAE(Y, Y_predict_skl)}, {MAE(Y, Y_predict)}')

MSE для двух моделей (sklearn и LinearRegressionByMatrix) соответственно:      27173382.577597994, 27173382.577598065
RMSE для двух моделей (sklearn и LinearRegressionByMatrix) соответственно:      5212.809470678742, 5212.80947067875
MAE для двух моделей (sklearn и LinearRegressionByMatrix) соответственно:      466.0181622241299, 466.0181622240003


**Видно, что результаты аналогичны.**